In [80]:
import numpy as np
import pandas as pd
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
import pickle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tqdm import tqdm

In [81]:
HOME_DIR = '/data/amandeep/nih-dataset/person'
table_path = f'{HOME_DIR}/tl_person_coinvestigator_with_qnihid.tsv'
f_name = table_path.split("/")[-1]
wikify_column_name = "person_name"
final_score_column = "siamese_prediction"

canonical_file_path = f'{HOME_DIR}/temp/canonical.csv'
candidate_file_path = f'{HOME_DIR}/temp/candidates.csv'
aux_field = 'graph_embedding_complex,class_count,property_count,context'
temp_dir= f'{HOME_DIR}/temp/temp'

aligned_pagerank_candidate_file_path = f'{HOME_DIR}/temp/apr_test.csv'
model_file_path = './models/weighted_lr.pkl'
ranking_model_file_path = './models/epoch_5_loss_0.09882864356040955_top1_0.8968926553672316.pth'
min_max_scaler_path = './models/normalization_factor.pkl'

model_voted_candidate_file_path = f'{HOME_DIR}/temp/mv_test.csv'
graph_embedding_file_path = f'{HOME_DIR}/temp/score_test.csv'

lof_reciprocal_rank_file_path = f'{HOME_DIR}/temp/lof_rr_test.csv'
lof_tfidf_file_path = f'{HOME_DIR}/temp/lof_tfidf_test.csv'
lof_feature_file = f'{HOME_DIR}/temp/lof_feature.csv'
context_score_file = f'{HOME_DIR}/temp/context_score_file.csv'

output_model_pred_file = f'{HOME_DIR}/temp/model_prediction.csv'
top5_links = f'{HOME_DIR}/temp/top5_links.csv'
colorized_kg_links = f'{HOME_DIR}/temp/{f_name.strip(".csv")}_colorized.xlsx'

graph_embedding_complex_file = f'{HOME_DIR}/temp/graph_embedding_complex.tsv'
class_count_file = f'{HOME_DIR}/temp/class_count.tsv'
property_count_file = f'{HOME_DIR}/temp/property_count.tsv'
context_file = f'{HOME_DIR}/temp/context.tsv'
index_url = 'http://ckg07:9200/wikidatadwd-augmented/'

string_threshold = 0.9
siamese_threshold = 0.9


In [82]:
features = ['pagerank','retrieval_score','monge_elkan','monge_elkan_aliases','des_cont_jaccard',
            'jaro_winkler','levenshtein','singleton','num_char','num_tokens',
           'lof_class_count_tf_idf_score', 'lof_property_count_tf_idf_score',
           'lof-graph-embedding-score', 'lof-reciprocal-rank', 'context_score']

In [83]:
print(len(features))

15


In [84]:
!ls "$table_path"

/data/amandeep/nih-dataset/person/tl_person_coinvestigator_with_qnihid.tsv


### Canonicalize

In [13]:
!tl canonicalize -c "$wikify_column_name" --add-context "$table_path" -s person_qnihid --tsv \
> "$canonical_file_path"

canonicalize Time: 0.3060629367828369s


In [14]:
pd.read_csv(canonical_file_path, nrows = 5)

,column,row,label,context,filename,column-id
0,0,0,LINDLEY BARBEE,UNIVERSITY OF WASHINGTON|CHRISTINE MITRA KHOSR...,tl_person_coinvestigator_with_qnihid.tsv,tl_person_coinvestigator_with_qnihid.tsv-0
1,0,1,CHRISTINE MITRA KHOSROPOUR,UNIVERSITY OF WASHINGTON|LINDLEY BARBEE,tl_person_coinvestigator_with_qnihid.tsv,tl_person_coinvestigator_with_qnihid.tsv-0
2,0,2,MICHAEL J GALE,UNIVERSITY OF WASHINGTON|CAROLYN B COYNE;HUGO ...,tl_person_coinvestigator_with_qnihid.tsv,tl_person_coinvestigator_with_qnihid.tsv-0
3,0,3,DANIEL M. RATNER,UNIVERSITY OF WASHINGTON|PATRICK S. STAYTON;SH...,tl_person_coinvestigator_with_qnihid.tsv,tl_person_coinvestigator_with_qnihid.tsv-0
4,0,4,SHAWN J. SKERRETT,UNIVERSITY OF WASHINGTON|COURTNEY CRANE;DANIEL...,tl_person_coinvestigator_with_qnihid.tsv,tl_person_coinvestigator_with_qnihid.tsv-0


In [15]:
!ls -l $canonical_file_path

-rw-r--r-- 1 amandeep isdstaff 9682890 Jul 27 16:34 /data/amandeep/nih-dataset/temp/canonical.csv


In [16]:
def split(f_path, output_path):
    df = pd.read_csv(f_path)
    d_list = np.array_split(df, 50)
    for i, d in enumerate(d_list):
        d.to_csv(f'{output_path}/split_{i}.csv', index=False)

In [17]:
split(canonical_file_path, '/data/amandeep/nih-dataset/person/nih-split')

# Candidate Generation

In [ ]:
!tl clean -c label -o label_clean "$canonical_file_path" \
/ --url http://ckg07:9200 --index wikidatadwd-augmented \
get-fuzzy-augmented-matches -c label_clean \
--auxiliary-fields "$aux_field" \
--auxiliary-folder "$temp_dir" \
/ --url http://ckg07:9200 --index wikidatadwd-augmented \
get-exact-matches \
-c label_clean --auxiliary-fields "$aux_field" \
--auxiliary-folder "$temp_dir" > "$candidate_file_path"

In [18]:
def candidate_generation(path, output_path, class_count_path, prop_count_path, context_path, graph_embedding):
    file_list = glob.glob(path + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        st = time.time()
        filename = file.split('/')[-1]
        print(f"{filename}: {i+1} of {len(file_list)}")
        output_file = f"{output_path}/{filename}"
        
        !tl clean -c label -o label_clean $file / \
        --url http://ckg07:9200 --index wikidatadwd-augmented \
        get-fuzzy-augmented-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" / \
        --url http://ckg07:9200 --index wikidatadwd-augmented \
        get-exact-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" > "$output_file"
        
        for field in aux_field.split(','):
            aux_list = []
            for f in glob.glob(f'{temp_dir}/*{field}.tsv'):
                aux_list.append(pd.read_csv(f, sep='\t', dtype=object))
            aux_df = pd.concat(aux_list).drop_duplicates(subset=['qnode'])
            if field == 'class_count':
                class_count_file = f"{class_count_path}/{filename[:-4]}_class_count.tsv"
                aux_df.to_csv(class_count_file, sep='\t', index=False)
            elif field == 'property_count':
                prop_count_file = f"{prop_count_path}/{filename[:-4]}_prop_count.tsv"
                aux_df.to_csv(prop_count_file, sep='\t', index=False)
            elif field == 'context':
                context_file = f"{context_path}/{filename[:-4]}_context.tsv"
                aux_df.to_csv(context_file, sep='\t', index=False)
            else:
                graph_embedding_file = f"{graph_embedding}/{filename[:-4]}_graph_embedding_complex.tsv"
                aux_df.to_csv(graph_embedding_file, sep='\t', index=False)

In [19]:
input_path = '/data/amandeep/nih-dataset/person/nih-split'
output_path = '/data/amandeep/nih-dataset/person/candidates'
class_count_path = '/data/amandeep/nih-dataset/person/temp/class_c'
prop_count_path = '/data/amandeep/nih-dataset/person/temp/prop_c'
context_path = '/data/amandeep/nih-dataset/temp/person/context'
graph_embedding = '/data/amandeep/nih-dataset/person/temp/ge'
import time

In [20]:
candidate_generation(input_path, output_path, class_count_path, prop_count_path, context_path, graph_embedding)

0it [00:00, ?it/s]

split_0.csv: 1 of 50
clean Time: 0.052808523178100586s
get-fuzzy-augmented-matches Time: 201.6638102531433s
get-exact-matches Time: 11.67469573020935s


1it [04:06, 246.90s/it]

split_1.csv: 2 of 50
clean Time: 0.055995941162109375s
get-fuzzy-augmented-matches Time: 245.99158239364624s
get-exact-matches Time: 10.243574619293213s


2it [08:58, 273.25s/it]

...

48it [1:48:27, 104.26s/it]

split_48.csv: 49 of 50
clean Time: 0.09058356285095215s
get-fuzzy-augmented-matches Time: 67.15201926231384s
get-exact-matches Time: 4.9366490840911865s


49it [1:50:02, 101.68s/it]

split_49.csv: 50 of 50
clean Time: 0.08889007568359375s
get-fuzzy-augmented-matches Time: 73.63883328437805s
get-exact-matches Time: 5.12647271156311s


50it [1:51:45, 134.11s/it]


In [21]:
features_path = '/data/amandeep/nih-dataset/person/features'
classifier_features= ['aligned_pagerank', 'smallest_qnode_number', 'monge_elkan', 'des_cont_jaccard_normalized']

In [22]:
def feature_generation(candidate_dir, embedding_dir, class_count_dir, property_count_dir, context_path, output_path):
    file_list = glob.glob(candidate_dir + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        filename = file.split('/')[-1]
        print(f"{filename}: {i+1} of {len(file_list)}")
        embedding_file = f"{embedding_dir}/{filename[:-4]}_graph_embedding_complex.tsv"
        class_count_file = f"{class_count_dir}/{filename[:-4]}_class_count.tsv"
        property_count_file = f"{property_count_dir}/{filename[:-4]}_prop_count.tsv"
        context_file = f"{context_path}/{filename[:-4]}_context.tsv"
        output_file = f"{output_path}/{filename}"
        if os.path.getsize(file) == 0:
                continue
        classifier_features_str = ",".join(classifier_features)
        !tl align-page-rank "$file" \
            / string-similarity -i --method symmetric_monge_elkan:tokenizer=word -o monge_elkan \
            / string-similarity -i --method symmetric_monge_elkan:tokenizer=word -c label_clean kg_aliases -o monge_elkan_aliases \
            / string-similarity -i --method jaro_winkler -o jaro_winkler \
            / string-similarity -i --method levenshtein -o levenshtein \
            / string-similarity -i --method jaccard:tokenizer=word -c kg_descriptions context -o des_cont_jaccard \
            / normalize-scores -c des_cont_jaccard / smallest-qnode-number \
            / mosaic-features -c kg_labels --num-char --num-tokens \
            / create-singleton-feature -o singleton \
            / vote-by-classifier  \
            --prob-threshold 0.995 \
            --features "$classifier_features_str" \
            --model "$model_file_path" \
            / score-using-embedding \
            --column-vector-strategy centroid-of-lof \
            --lof-strategy ems-mv \
            -o lof-graph-embedding-score \
            --embedding-file "$embedding_file" \
            / generate-reciprocal-rank  \
            -c lof-graph-embedding-score \
            -o lof-reciprocal-rank \
            / compute-tf-idf  \
            --feature-file "$class_count_file" \
            --feature-name class_count \
            --singleton-column is_lof \
            -o lof_class_count_tf_idf_score \
            / compute-tf-idf \
            --feature-file "$property_count_file" \
            --feature-name property_count \
            --singleton-column is_lof \
            -o lof_property_count_tf_idf_score   > "$output_file"

In [23]:
feature_generation(output_path, graph_embedding, class_count_path, prop_count_path, context_path, features_path)

0it [00:00, ?it/s]

split_0.csv: 1 of 50
align-page-rank Time: 2.2507216930389404s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 22.784616470336914s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 77.4554123878479s
string-similarity-['jaro_winkler'] Time: 5.552843809127808s
string-similarity-['levenshtein'] Time: 20.565762042999268s
string-similarity-['jaccard:tokenizer=word'] Time: 1.417466640472412s
normalize-scores-des_cont_jaccard Time: 0.43269824981689453s
smallest-qnode-number Time: 3.8840510845184326s
mosaic-features Time: 0.19453907012939453s
create-singleton-feature Time: 2.8804738521575928s
vote-by-classifier Time: 5.029954195022583s
Qnodes to lookup: 102630
Qnodes from file: 102203
Outlier removal generates 298 lof-voted candidates
score-using-embedding Time: 195.41063594818115s
generate-reciprocal-rank-lof-graph-embedding-score Time: 1.4363844394683838s
compute-tf-idf-class_count Time: 211.1447446346283s
compute-tf-idf-property_count Time: 217.32307767868042

1it [03:45, 225.67s/it]

split_1.csv: 2 of 50
align-page-rank Time: 2.032576322555542s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 22.90036940574646s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 77.6394567489624s
string-similarity-['jaro_winkler'] Time: 5.873132705688477s
string-similarity-['levenshtein'] Time: 20.538536310195923s
string-similarity-['jaccard:tokenizer=word'] Time: 1.3488471508026123s
normalize-scores-des_cont_jaccard Time: 0.41242361068725586s
smallest-qnode-number Time: 3.68790340423584s
mosaic-features Time: 0.19437146186828613s
create-singleton-feature Time: 2.4444289207458496s
vote-by-classifier Time: 2.2058184146881104s
Qnodes to lookup: 102340
Qnodes from file: 101891
Outlier removal generates 287 lof-voted candidates
score-using-embedding Time: 180.6338300704956s
generate-reciprocal-rank-lof-graph-embedding-score Time: 1.4756865501403809s
compute-tf-idf-class_count Time: 193.07104015350342s
compute-tf-idf-property_count Time: 198.64530539512634s


2it [07:12, 214.52s/it]

...

48it [2:52:45, 211.42s/it]

split_48.csv: 49 of 50
align-page-rank Time: 2.014023780822754s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 22.608262300491333s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 76.41889429092407s
string-similarity-['jaro_winkler'] Time: 5.446140289306641s
string-similarity-['levenshtein'] Time: 20.56031084060669s
string-similarity-['jaccard:tokenizer=word'] Time: 1.6000192165374756s
normalize-scores-des_cont_jaccard Time: 0.42737436294555664s
smallest-qnode-number Time: 3.73067569732666s
mosaic-features Time: 0.19897246360778809s
create-singleton-feature Time: 2.4266467094421387s
vote-by-classifier Time: 1.4760348796844482s
Qnodes to lookup: 101851
Qnodes from file: 101432
Outlier removal generates 292 lof-voted candidates
score-using-embedding Time: 175.18187546730042s
generate-reciprocal-rank-lof-graph-embedding-score Time: 1.4361016750335693s
compute-tf-idf-class_count Time: 186.13545632362366s
compute-tf-idf-property_count Time: 191.731784105300

49it [2:56:05, 208.05s/it]

split_49.csv: 50 of 50
align-page-rank Time: 2.2183539867401123s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 23.31024169921875s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 75.28602361679077s
string-similarity-['jaro_winkler'] Time: 5.629039525985718s
string-similarity-['levenshtein'] Time: 20.43037748336792s
string-similarity-['jaccard:tokenizer=word'] Time: 1.3339831829071045s
normalize-scores-des_cont_jaccard Time: 0.39553332328796387s
smallest-qnode-number Time: 3.7197887897491455s
mosaic-features Time: 0.19759726524353027s
create-singleton-feature Time: 2.4863481521606445s
vote-by-classifier Time: 1.5447382926940918s
Qnodes to lookup: 103643
Qnodes from file: 103224
Outlier removal generates 312 lof-voted candidates
score-using-embedding Time: 174.2413935661316s
generate-reciprocal-rank-lof-graph-embedding-score Time: 1.4542596340179443s
compute-tf-idf-class_count Time: 185.37823343276978s
compute-tf-idf-property_count Time: 190.61177301406

50it [2:59:24, 215.30s/it]


In [17]:
def concat_files(i_path, output_file, sep=None):
    df_l = []
    for f in glob.glob(f'{i_path}/*'):
        if sep:
            df_l.append(pd.read_csv(f, sep=sep))
        else:
            df_l.append(pd.read_csv(f))
    if sep:
        pd.concat(df_l).to_csv(output_file, index=False, sep=sep)
    else:
        pd.concat(df_l).to_csv(output_file, index=False)

In [6]:
def run_context_match(features_path, context_path, custom_file, output_path):
    file_list = glob.glob(features_path + '/*.csv')
    for i, f in tqdm(enumerate(file_list)):
        if i > 26:
            f_name = f.split('/')[-1]
            print(f'{f_name}: {i+1} of {len(file_list)}')
            context_file = f"{context_path}/{f_name[:-4]}_context.tsv"
            output_file = f"{output_path}/{f_name}"
            !tl context-match --custom-context-file $custom_file \
            --context-file $context_file --string-separator ";" \
            --similarity-string-threshold $string_threshold $f > $output_file
        

In [7]:
context_path = '/data/amandeep/nih-dataset/temp/context'
features_path = '/data/amandeep/nih-dataset/features'
context_output_path = '/data/amandeep/nih-dataset/features_with_context'

In [8]:
run_context_match(features_path, context_path, '/data/amandeep/nih-dataset/coauthors.context.tsv.gz', context_output_path)

0it [00:00, ?it/s]

split_27.csv: 28 of 50


28it [26:14, 56.23s/it]

split_28.csv: 29 of 50


29it [53:29, 132.88s/it]

...

48it [9:23:19, 1611.22s/it]

split_48.csv: 49 of 50


49it [9:53:12, 1665.37s/it]

split_49.csv: 50 of 50


50it [10:20:31, 744.64s/it] 


In [9]:
prediction_path = '/data/amandeep/nih-dataset/person/predictions'

In [10]:
features_str = ",".join(features)
def run_prediction(features_path, prediction_path):
    file_list = glob.glob(features_path + '/*.csv')
    for i, f in tqdm(enumerate(file_list)):
        f_name = f.split('/')[-1]
        print(f'{f_name}: {i+1} of {len(file_list)}')
        output_file = f"{prediction_path}/{f_name}"
        !tl predict-using-model -o siamese_prediction \
        --ranking-model $ranking_model_file_path \
        --features $features_str \
        --normalization-factor $min_max_scaler_path $f > $output_file

In [11]:
run_prediction(context_output_path, prediction_path)

0it [00:00, ?it/s]

split_0.csv: 1 of 50
/data/amandeep/github/table-linker/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
predict-using-model Time: 9.645196199417114s


1it [01:08, 68.19s/it]

split_1.csv: 2 of 50
/data/amandeep/github/table-linker/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
predict-using-model Time: 7.076881408691406s


2it [01:23, 36.96s/it]

...

48it [19:35, 19.17s/it]

split_48.csv: 49 of 50
/data/amandeep/github/table-linker/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
predict-using-model Time: 7.273780107498169s


49it [19:50, 17.94s/it]

split_49.csv: 50 of 50
/data/amandeep/github/table-linker/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
predict-using-model Time: 8.205457925796509s


50it [20:05, 24.12s/it]


In [12]:
colorized_path = '/data/amandeep/nih-dataset/person/colorized'

In [13]:
def topk_color(prediction_path, colorized_path):
    file_list = glob.glob(prediction_path + '/*.csv')
    for i, f in tqdm(enumerate(file_list)):
        f_name = f.split('/')[-1]
        print(f'{f_name}: {i+1} of {len(file_list)}')
        output_file = f"{colorized_path}/{f_name[:-4]}.xlsx"
        !tl get-kg-links -c $final_score_column -k 5 --k-rows $f \
        / add-color -c "$final_score_column" -k 5 --output "$output_file"

In [14]:
topk_color(prediction_path, colorized_path)

0it [00:00, ?it/s]

split_0.csv: 1 of 50
get-kg-links-siamese_prediction Time: 6.869730472564697s
add-color Time: 1.7595765590667725s


1it [00:15, 15.27s/it]

split_1.csv: 2 of 50
get-kg-links-siamese_prediction Time: 5.5476555824279785s
add-color Time: 1.4353773593902588s


2it [00:28, 14.20s/it]

...

48it [11:20, 14.62s/it]

split_48.csv: 49 of 50
get-kg-links-siamese_prediction Time: 5.665173768997192s
add-color Time: 1.4994769096374512s


49it [11:34, 14.51s/it]

split_49.csv: 50 of 50
get-kg-links-siamese_prediction Time: 5.610145568847656s
add-color Time: 1.451505184173584s


50it [11:48, 14.16s/it]


In [25]:
def add_NILS(colorized_path, output_path):
    file_list = glob.glob(colorized_path + '/*.xlsx')
    for i, f in tqdm(enumerate(file_list)):
        f_name = f.split('/')[-1]
        print(f'{f_name}: {i+1} of {len(file_list)}')
        output_file = f"{output_path}/{f_name[:-5]}.csv"
        
        df = pd.read_excel(f)
        df.loc[df['siamese_prediction'].astype(float) < 0.9, 'kg_id'] = 'NIL'
        df.to_csv(output_file, index=False)

In [18]:
nil_path = '/data/amandeep/nih-dataset/person/with_nils'
add_NILS(colorized_path, nil_path)

0it [00:00, ?it/s]

split_0.xlsx: 1 of 50


1it [00:03,  3.12s/it]

split_1.xlsx: 2 of 50


2it [00:06,  3.14s/it]

...

48it [02:10,  2.54s/it]

split_48.xlsx: 49 of 50


49it [02:12,  2.52s/it]

split_49.xlsx: 50 of 50


50it [02:15,  2.71s/it]


In [26]:
def count_non_nils(nil_path):
    file_list = glob.glob(nil_path + '/*.csv')
    o = list()
    for i, f in tqdm(enumerate(file_list)):
        f_name = f.split('/')[-1]
        o.append(pd.read_csv(f))
    df = pd.concat(o)
    print(len(df))
    df = df[df['kg_id'] != 'NIL']
    print(len(df))

        

In [27]:
count_non_nils(nil_path)

50it [00:00, 120.06it/s]

51568
21910


In [33]:
def join(nil_path):
    file_list = glob.glob(nil_path + '/*.csv')
    o = list()
    for i, f in tqdm(enumerate(file_list)):
        f_name = f.split('/')[-1]
        o.append(pd.read_csv(f))
    df = pd.concat(o)
    df.to_csv('/tmp/joined.csv', index=False)

In [34]:
join(nil_path)

50it [00:06,  7.86it/s]


In [36]:
pd.read_csv(table_path, sep='\t').to_csv(table_path[:-4] + '.csv', index=False)

In [38]:
!tl join -c siamese_prediction -f /data/amandeep/nih-dataset/person/tl_person_coinvestigator_with_qnihid.csv \
--extra-info /tmp/joined.csv > '/data/amandeep/nih-dataset/person/tl_person_coinvestigator_with_qnihid_joined.csv'

join Time: 98.23094987869263s


In [78]:
def create_replace_nodes_mapping(joined_file, mapping_file):
    df = pd.read_csv(joined_file)
    essential_columns = ['person_qnihid', 'person_name_kg_id', 'person_name_score']
    cols = df.columns
    for c in cols:
        if c not in essential_columns:
            df = df.drop(columns=c)
    df['label'] = 'same_as_item'
    df.rename(columns={'person_qnihid': 'node1', 'person_name_kg_id': 'node2', 'person_name_score':'confidence'}, inplace=True)
    df = df[['node1', 'label', 'node2', 'confidence']]
    df.drop_duplicates(subset=['node1', 'node2'], inplace=True)
    oo = []
    for _, gdf in df.groupby('node1'):
        gdf = gdf[gdf['node2'] != 'NIL']
        if len(gdf) > 1:
            oo.append(pd.DataFrame(gdf.head(0)))
        else:
            oo.append(gdf)
    pd.concat(oo).to_csv(mapping_file, sep='\t', index=False)

In [79]:
create_replace_nodes_mapping('/data/amandeep/nih-dataset/person/tl_person_coinvestigator_with_qnihid_joined.csv', '/data/amandeep/nih-dataset/person/replace_nodes_mapping.tsv')

In [60]:
def concat_person_files():
    f_p = "/data/amandeep/nih-dataset/person/kgtk-files-nih V2.0"
    o = []
    for f in glob.glob(f"{f_p}/*tsv"):
        o.append(pd.read_csv(f, sep='\t'))
    df = pd.concat(o)
    df.to_csv('/data/amandeep/nih-dataset/person/nih-person-kgtk.tsv', sep='\t', index=False)

In [61]:
concat_person_files()

kgtk replace-nodes -i nih-person-kgtk.tsv -o nih-person-kgtk-wikidata-qnodes.tsv --mapping-file replace_nodes_mapping.tsv